In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
import pandas as pd
import numpy as np
import os

### QUOTES PRICES

In [18]:
fund = run.load_pkl(f'{b3.app_folder}/fund')
quotes = run.load_pkl(f'{b3.app_folder}/quotes')


In [19]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

(dict_keys(['BENS INDUSTRIAIS', 'COMUNICACOES', 'CONSUMO CICLICO', 'CONSUMO NAO CICLICO', 'FINANCEIRO', 'MATERIAIS BASICOS', 'NAO CLASSIFICADOS', 'Nenhum', 'OUTROS', 'PETROLEO GAS E BIOCOMBUSTIVEIS', 'SAUDE', 'TECNOLOGIA DA INFORMACAO', 'UTILIDADE PUBLICA']),
 Index(['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO',
        'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
        'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA',
        'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF', 'COMPANHIA',
        'PREGAO', 'TICK', 'LISTAGEM', 'CVM', 'TICKERS', 'ISIN', 'CNPJ',
        'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE', 'ESCRITURADOR',
        'CD_CONTA_original', 'DS_CONTA_original'],
       dtype='object'))

In [20]:
quotes.keys(), quotes['AERIS'].keys()

(dict_keys(['AERIS', 'ESTAPAR', 'ARMAC', 'ATMASA', 'AZEVEDO', 'AZUL', 'BARDELLA', 'BBMLOGISTICA', 'CCR SA', 'GRUAIRPORT', 'ECOVIAS', 'CONC RIO TER', 'ROD TIETE', 'DTCOMDIRECT', 'ECORODOVIAS', 'ACO ALTONA', 'EMBPAR SA', 'EMBRAER', 'ETERNIT', 'FER C ATLANT', 'FLEX SA', 'FRASLE', 'GOL', 'GPS', 'HAGA SA', 'HIDROVIAS', 'HMOBI SA', 'ROMI', 'INEPAR', 'INVEPAR', 'JSL', 'KEPLER WEBER', 'LOGIN', 'MARCOPOLO', 'METALFRIO', 'RIOSULENSE', 'METISA', 'MILLS', 'MINASMAQUINA', 'MRS LOGIST', 'NORDON MET', 'PORTOBELLO', 'PORTO VM', 'PRATICA', 'PRINER', 'RANDON PART', 'RECRUSUL', 'ALL NORTE', 'RUMO SA', 'SALUS INFRA', 'SANTOS BRP', 'SCHULZ', 'SEQUOIA LOG', 'SONDOTECNICA', 'TAURUS ARMAS', 'TEGMA', 'TRIUNFO PART', 'TREVISA', 'VALID', 'WEG', 'WETZEL SA', 'WILSON SONS', 'WLM IND COM', 'ALGAR TELEC', 'BRISANET', 'BRITANIA', 'DESKTOP', 'TELEBRAS', 'TELEF BRASIL', 'UNIFIQUE', 'ALLIED', 'ALPARGATAS', 'ALPHAVILLE', 'AMERICANAS', 'ANIMA', 'AREZZO CO', 'BAHEMA', 'BAHIAPCH', 'BIC MONARK', 'ZAMP SA', 'CEA MODAS', 'CEAB

In [29]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.remaining_time(start_time, len(fund), i))


BENS INDUSTRIAIS 7.69% 1+12, 37.591383s per item, Remaining: 0h 07m 31s
COMUNICACOES 15.38% 2+11, 20.503914s per item, Remaining: 0h 03m 45s
CONSUMO CICLICO 23.08% 3+10, 31.323545s per item, Remaining: 0h 05m 13s
CONSUMO NAO CICLICO 30.77% 4+9, 27.105038s per item, Remaining: 0h 04m 03s
FINANCEIRO 38.46% 5+8, 29.310160s per item, Remaining: 0h 03m 54s
MATERIAIS BASICOS 46.15% 6+7, 26.920314s per item, Remaining: 0h 03m 08s
NAO CLASSIFICADOS 53.85% 7+6, 24.885268s per item, Remaining: 0h 02m 29s
Nenhum 61.54% 8+5, 22.489914s per item, Remaining: 0h 01m 52s
OUTROS 69.23% 9+4, 20.515935s per item, Remaining: 0h 01m 22s
PETROLEO GAS E BIOCOMBUSTIVEIS 76.92% 10+3, 18.880425s per item, Remaining: 0h 00m 56s
SAUDE 84.62% 11+2, 17.803337s per item, Remaining: 0h 00m 35s
TECNOLOGIA DA INFORMACAO 92.31% 12+1, 16.601552s per item, Remaining: 0h 00m 16s
UTILIDADE PUBLICA 100.00% 13+0, 18.309642s per item, Remaining: 0h 00m 00s


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.remaining_time(start_time, len(df_preplot), i))


BENS INDUSTRIAIS 7.69% 1+12, 2.123899s per item, Remaining: 0h 00m 25s
COMUNICACOES 15.38% 2+11, 1.111040s per item, Remaining: 0h 00m 12s
CONSUMO CICLICO 23.08% 3+10, 1.014373s per item, Remaining: 0h 00m 10s
CONSUMO NAO CICLICO 30.77% 4+9, 0.938078s per item, Remaining: 0h 00m 08s
FINANCEIRO 38.46% 5+8, 0.803704s per item, Remaining: 0h 00m 06s
MATERIAIS BASICOS 46.15% 6+7, 0.682425s per item, Remaining: 0h 00m 04s
NAO CLASSIFICADOS 53.85% 7+6, 0.624585s per item, Remaining: 0h 00m 03s
Nenhum 61.54% 8+5, 0.569693s per item, Remaining: 0h 00m 02s
OUTROS 69.23% 9+4, 0.514146s per item, Remaining: 0h 00m 02s
PETROLEO GAS E BIOCOMBUSTIVEIS 76.92% 10+3, 0.469080s per item, Remaining: 0h 00m 01s
SAUDE 84.62% 11+2, 0.432391s per item, Remaining: 0h 00m 00s
TECNOLOGIA DA INFORMACAO 92.31% 12+1, 0.398593s per item, Remaining: 0h 00m 00s
UTILIDADE PUBLICA 100.00% 13+0, 0.433056s per item, Remaining: 0h 00m 00s


In [37]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

,Open,High,Low,Close,Adj Close,Volume,PREGAO,TICKER,DT_REFER,FILENAME,...,18.09 - Remuneração de Capital de Terceiros por Valor Agregado,18.09.01 - Juros Pagos a Terceiros por Valor Agregado,18.09.02 - Aluguéis Pagos a Terceiros por Valor Agregado,18.10 - Remuneração de Capital Próprio por Valor Agregado,18.10.01 - Juros Sobre Capital Próprio por Valor Agregado,18.10.02 - Dividendos por Valor Agregado,18.10.02 - Lucros Retidos por Valor Agregado,"18.11.01 - Alíquota de Impostos (Impostos, Taxas e Contribuições por Receita Bruta)",18.11.02 - Taxa de Juros Pagos (Remuneração de Capital de Terceiros por Receita Bruta,18.11.03 - Taxa de Proventos Gerados (Remuneração de Capital Próprio por Receita Bruta
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-07,0.01,0.01,0.01,0.01,0.006135,24000.0,ACO ALTONA,AERI3,2011-01-03,dfp_cia_aberta_BPA_ind_2010.csv,...,-1.183290,0.108827,NaN,0.436596,NaN,NaN,0.067577,0.202823,-0.711380,0.262477
2011-03-31,4.50,4.50,4.50,4.50,3.298934,NaN,ACO ALTONA,AERI3,2011-03-31,itr_cia_aberta_BPA_ind_2011.csv,...,-0.633810,0.109532,NaN,0.459878,NaN,NaN,0.089663,0.169332,-0.333927,0.242290
2011-06-30,4.50,4.50,4.50,4.50,3.298934,NaN,ACO ALTONA,AERI3,2011-06-30,itr_cia_aberta_BPA_ind_2011.csv,...,-0.540203,0.103000,NaN,0.517282,NaN,NaN,0.115173,0.134906,-0.295050,0.282531
2011-09-30,4.50,4.50,4.50,4.50,3.298934,NaN,ACO ALTONA,AERI3,2011-09-30,itr_cia_aberta_BPA_ind_2011.csv,...,-1.262985,0.144619,NaN,0.780793,NaN,NaN,0.206301,-0.039463,-0.456184,0.282019
2012-01-02,4.50,4.50,4.50,4.50,3.298934,NaN,ACO ALTONA,AERI3,2012-01-02,dfp_cia_aberta_BPA_ind_2011.csv,...,-2.288831,0.149682,NaN,0.272214,NaN,NaN,0.017466,0.347901,-1.721434,0.204733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,25.00,25.00,25.00,25.00,23.911798,NaN,WLM IND COM,AERI3,2022-06-30,itr_cia_aberta_BPA_con_2022.csv,...,-4.873302,0.047274,NaN,0.732878,0.037161,NaN,0.350724,0.053903,-0.937574,0.140998
2022-09-30,20.01,20.01,20.01,20.01,19.205019,NaN,WLM IND COM,AERI3,2022-09-30,itr_cia_aberta_BPA_con_2022.csv,...,-6.782529,0.063359,NaN,0.729876,0.054790,NaN,0.298275,0.040819,-0.963579,0.103692
2023-01-02,21.50,21.50,21.50,21.50,20.755247,NaN,WLM IND COM,AERI3,2023-01-02,dfp_cia_aberta_BPA_con_2022.csv,...,-6.228644,0.114816,NaN,0.765106,0.099840,0.059036,0.165835,0.047293,-0.953232,0.117092


### DASH

In [63]:
company = 'MAGAZ LUIZA'
df = run.load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')

# df.columns.to_list()

In [64]:
quote_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
new_cols = {col: f"50.{i:02} - {col}" for i, col in enumerate(quote_cols, start=1)}
df.rename(columns=new_cols, inplace=True)


In [66]:
new_cols

{'Open': '50.01 - Open',
 'High': '50.02 - High',
 'Low': '50.03 - Low',
 'Close': '50.04 - Close',
 'Adj Close': '50.05 - Adj Close',
 'Volume': '50.06 - Volume'}

In [61]:
num_cols = [col for col in df.columns if col[0].isdigit()]
df[num_cols] = df[num_cols].astype('float64')


In [38]:
num_cols = [col for col in df.columns if col[0].isdigit()]
float_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
df[num_cols+float_cols] = df[num_cols+float_cols].astype('float64')
df = df.copy()

In [5]:
import assets.graphs as g
# Get default graphs
default_graphs = g.get_default_graphs(df)
default_graphs


{'01': {0: {0: {'info': {'title': '01 - Ativo Total',
     'header': 'Totalidade dos Bens e Direitos',
     'description': 'O Ativo Total representa a soma de todos os bens e direitos que uma empresa possui em um determinado momento. Valores ascendentes podem indicar crescimento e expansão da empresa, enquanto declínios podem sinalizar venda de ativos ou redução das operações.',
     'footer': 'O Ativo Total é a consolidação de todos os recursos controlados por uma entidade, resultantes de eventos passados e dos quais se espera que a entidade obtenha benefícios econômicos futuros. É a combinação dos ativos circulantes (aqueles que serão convertidos em dinheiro ou consumidos no ciclo operacional) e dos ativos não circulantes (bens e direitos que não serão convertidos em dinheiro no curto prazo). Pense no Ativo Total como uma grande caixa onde uma empresa armazena tudo o que possui, desde dinheiro em caixa até prédios e maquinários.'},
    'data': {'axis': ['Reais (RS)', 'Porcentagem (%)

In [11]:
default_graphs.keys()

dict_keys(['01'])

In [35]:
manual_entries = {
    '99': {
        0: {
            0: {
                'info': {
                    'title': '99 - PVPA',
                    'header': '99 - PVPA',
                    'description': '99 - PVPA',
                    'footer': '99 - PVPA'
                },
                'data': {
                    'axis': ['Reais (RS)', 'Porcentagem (%)'],
                    'left': ['99 - PVPA']
                },
                'options': {
                    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False}
                }
            },
            1: {
                'info': {
                    'title': '02.03 - Patrimônio Líquido',
                    'header': '99 - Example',
                    'description': '99 - Example',
                    'footer': '99 - Example'
                },
                'data': {
                    'axis': ['Reais (RS)', 'Porcentagem (%)'],
                    'left': ['02.03 - Patrimônio Líquido']
                },
                'options': {
                    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False}
                }
            }
        }
    },
    '98': {
        0: {
            0: {
                'info': {
                    'title': 'Adj Close',
                    'header': '98 - Sample',
                    'description': 'This is a sample for 98',
                    'footer': 'Footer for 98 sample'
                },
                'data': {
                    'axis': ['Reais (RS)', 'Porcentagem (%)'],
                    'left': ['Adj Close']
                },
                'options': {
                    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False}
                }
            },
            1: {
                'info': {
                    'title': '02.03 - Patrimônio Líquido',
                    'header': '98 - Example',
                    'description': 'This is another example for 98',
                    'footer': 'Footer for 98 example'
                },
                'data': {
                    'axis': ['Reais (RS)', 'Porcentagem (%)'],
                    'left': ['02.03 - Patrimônio Líquido']
                },
                'options': {
                    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False}
                }
            }
        }
    }
}


In [ ]:
df['Adj Close']

In [36]:
merged_entries = {**default_graphs, **manual_entries}
merged_entries.keys()


dict_keys(['01', '02', '99', '98'])

In [37]:
a = []
for g, (group_key, group) in enumerate(merged_entries.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            run.plot_tweak(plot_info, df).show();
